<a href="https://colab.research.google.com/github/vineethvc/googleColab/blob/main/Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the drive and mount it to allow the notebook to access the Google drive for training data.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/Yolo/


/content/gdrive/My Drive/Yolo


In [ ]:
!ls

darknet  Logs  test.gdoc  Yolo.ipynb


Clone the YOLOv4 repository created by AlexeyAB from the below link. 

In [ ]:
! git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 13781, done.
remote: Total 13781 (delta 0), reused 0 (delta 0), pack-reused 13781
Receiving objects: 100% (13781/13781), 12.37 MiB | 3.16 MiB/s, done.
Resolving deltas: 100% (9405/9405), done.
Checking out files: 100% (2006/2006), done.


In [ ]:
%cd darknet/

/content/gdrive/My Drive/Yolo/darknet


Build the repository with 'make' command to create the executable file for darknet framework. Darknet framework allows to train an algorithm proposed by AlexeyAB in YOLOv4 paper and run predictions on images. 

In [ ]:
!make
!chmod +x ./darknet

chmod +x *.sh


In [ ]:
import tensorflow as tf 
tf.test.gpu_device_name() 

'/device:GPU:0'

Create a folder structure for log files and plot the accuracy and loss in the chart

In [ ]:
DRIVE_SAVE_DIR = '../Logs'

!mkdir -p "{DRIVE_SAVE_DIR}"
!rm -f "{DRIVE_SAVE_DIR}"/train.log
!touch "{DRIVE_SAVE_DIR}"/chart.png
!rm -f chart.png
!ln -sf "{DRIVE_SAVE_DIR}"/chart.png .
!mkdir -p "{DRIVE_SAVE_DIR}"/backup
!rm -rf backup
!ln -sf "{DRIVE_SAVE_DIR}"/backup .

Train the model with configurations stored in yolo-obj.cfg and the data fields stored in obj.data file. The weights file are downloaded from:
 
"https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137"



In [ ]:
! ./darknet detector train data/obj.data cfg/yolo-obj.cfg yolov4.conv.137 -dont_show -gpus 0 -map -dont_show 2>&1 | tee "{DRIVE_SAVE_DIR}"/train.log | grep -E "hours left|mean_average"

 1: 2117.052002, 2117.052002 avg loss, 0.000000 rate, 7.271930 seconds, 64 images, -1.000000 hours left
 2: 2117.196289, 2117.066406 avg loss, 0.000000 rate, 7.540600 seconds, 128 images, 47.389569 hours left
 3: 2118.789795, 2117.238770 avg loss, 0.000000 rate, 7.598168 seconds, 192 images, 47.252628 hours left
 4: 2120.417236, 2117.556641 avg loss, 0.000000 rate, 7.693021 seconds, 256 images, 47.079727 hours left
 5: 2117.599854, 2117.561035 avg loss, 0.000000 rate, 7.412606 seconds, 320 images, 46.943864 hours left
 6: 2120.319092, 2117.836914 avg loss, 0.000000 rate, 8.102277 seconds, 384 images, 46.653061 hours left
 7: 2119.349365, 2117.988281 avg loss, 0.000000 rate, 8.047903 seconds, 448 images, 46.530101 hours left
 8: 2120.661865, 2118.255615 avg loss, 0.000000 rate, 8.237270 seconds, 512 images, 46.341272 hours left
 9: 2120.558594, 2118.485840 avg loss, 0.000000 rate, 7.842348 seconds, 576 images, 46.168407 hours left
 10: 2118.192871, 2118.456543 avg loss, 0.000000 rate, 7

If training is stopped midway it can be restarted using the weights saved in the backup folder. The weights are stored iteratively for each 100 epochs. After 1000 epochs the weights are stored for each 1000 epoch

In [ ]:
! ./darknet detector train data/obj.data cfg/yolo-obj.cfg "{DRIVE_SAVE_DIR}"/backup/yolo-obj_last.weights -dont_show -gpus 0 -map -dont_show 2>&1 | tee "{DRIVE_SAVE_DIR}"/train.log | grep -E "hours left|mean_average"

 501: 2.531853, 2.531853 avg loss, 0.000063 rate, 7.090419 seconds, 32064 images, -1.000000 hours left
 502: 3.457050, 2.624373 avg loss, 0.000064 rate, 7.556836 seconds, 32128 images, 22.922391 hours left
 503: 4.264814, 2.788417 avg loss, 0.000064 rate, 7.791933 seconds, 32192 images, 22.968842 hours left
 504: 4.533044, 2.962880 avg loss, 0.000065 rate, 8.071419 seconds, 32256 images, 23.034413 hours left
 505: 4.883020, 3.154894 avg loss, 0.000065 rate, 7.904548 seconds, 32320 images, 23.116090 hours left
 506: 3.761218, 3.215526 avg loss, 0.000066 rate, 7.980238 seconds, 32384 images, 23.178820 hours left
 507: 4.874547, 3.381428 avg loss, 0.000066 rate, 8.416364 seconds, 32448 images, 23.161104 hours left
 508: 4.857673, 3.529053 avg loss, 0.000067 rate, 8.112578 seconds, 32512 images, 23.235151 hours left
 509: 4.252836, 3.601431 avg loss, 0.000067 rate, 8.083258 seconds, 32576 images, 23.338818 hours left
 510: 4.142152, 3.655503 avg loss, 0.000068 rate, 8.229634 seconds, 32640

In [ ]:
! ./darknet detector test data/obj.data cfg/yolo-obj.cfg yolo-obj_last.weights data/obj/test/QP58872-34-Berlin-10115.jpeg

/bin/bash: ./darknet: Is a directory


Mean Average Precision is calculated for the trained model for different IoU thresholds. Higher the IoU level, more precise the model is expected to be.

In [ ]:
DRIVE_SAVE_DIR = '../Logs'

! ./darknet detector map -iou_thresh 0.8 -points 0 data/obj.data cfg/yolo-obj.cfg "{DRIVE_SAVE_DIR}"/backup/yolo-obj_last.weights

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 rou